<img src="images/escudo_utfsm.gif" style="float:right;height:100px">
<img src="images/IsotipoDIisocolor.png" style="float:left;height:100px">
<center>

    <i><h2 style="font-family:serif;font-size:300%; text-align:center;color:#283A5B"> Tarea 0</h2></i>
    <h1 style="font-family:serif;font-size:200%; text-align:center;color:#4d4d4d"> <i> Redes Neuronales Artificiales - San Joaquín </i></h1>
    <h3 style = 'font-family:serif;font-size:120%'><i> Ignacio Loayza C. 201273604-8</i></h3>

</center>

<p>
<center>_Marzo 2018_ </center>
</p>

In [1]:
# encoding: utf-8
%matplotlib inline

# semilla
seed = hash("Ñanculef es mi pastor, nada me ha de faltar")%2^32

# Imports

#Neural Networks
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense


#numerico
import numpy as np
import pandas as pd

#Machine Learning

#Metricas

#Graficos
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_style("darkgrid")
import graphviz

# Warnings
import warnings
warnings.filterwarnings('ignore')

#Shallow copy
import copy

#preprocesamiento


#HTML incrustation
from IPython.display import display, HTML

display(HTML("""
<style>
p {
    color: black;
    font-family: Serif;
    font-size: 1.23em;
}
.output_png {
        display: table-cell;
        text-align: center;
        vertical-align: middle;
    }

</style>

"""))


Using TensorFlow backend.


<i><h2 style="font-family:serif;font-size:260%; text-align:center;color:#004d80"> Back-propagation from Scratch</h2></i>

<i><h3 style="font-family:serif;font-size:200%; text-align:left;color:#4d4d4d"> I) Entrenando una red FF mediante Back-propagation</h3></i>

Comenzaremos entrenando una red del tipo *Feed Forward* con arquitectura de dos capas ocultas, distribuidas de la forma  32:16 neuronas en las capas y *K* neuronas de salida.

In [87]:
## Clases necesarias

class Network:
    def __init__(self, n_inputs, neurons_each_layer, n_outputs):
        self.n_inputs = n_inputs
        self.neurons_each_layer = neurons_each_layer
        self.n_outputs = n_outputs
        
        # Inicializacion de las capas
        layers_array = []
        index = 0
        n_neurons_prev_layer = 0 # para la capa de input

        #input layer
        layers_array.append(Layer(n_inputs, 0, index))
        n_neurons_prev_layer = n_inputs

        #Hidden layers
        for layer_conf in neurons_each_layer:
            index+=1
            layers_array.append(Layer(layer_conf, n_neurons_prev_layer,index))
            n_neurons_prev_layer = layer_conf
            
        #Output layer
        layers_array.append(Layer(n_outputs, n_neurons_prev_layer, index))
        self.hidden_layers = layers_array
            
        
    def get_weights_matrix(layer_index):
        return self.hidden_layers[layer_index].get_weights_matrix()
    
    def get_n_inputs(self):
        return n_inputs
    def get_neurons_each_layer(self):
        return self.neurons_each_layer
    def get_n_outputs(self):
        return n_outputs

    def get_network_parameters(self):
        network_params = {'n_inputs':self.n_inputs,
                        'neurons_each_layer':self.neurons_each_layer,
                        'n_outputs':self.n_outputs,
                         'hidden_layers':self.hidden_layers}
        return network_params
    
    def forward_propagate(self, data_row):
        previous_activations = np.ones((self.n_inputs,1))
        
        for layer in self.hidden_layers:
            if(layer.get_index() == 0):#Si es la primera capa le tenemos que pasar la data_column como las activaciones
                layer.compute_layer_activations(data_row)
                previous_activations = layer.get_activations_column()
            else:
                layer.compute_layer_activations(previous_activations)
                previous_activations = layer.get_activations_column()
                
    def backward_pass(self, ):
    
    
        # expected es un vector con las mismas dimensiones
        #que la columna de activaciones de la capa de salida.
        #Usemos el error cuadratico medio (MSE)
    def calulate_loss(self, expected): 
        predicted = self.hidden_layers[-1].get_activations_column
        return np.pow(expected - predicted,2)/2
    def gradient_loss(self, expected):
        predicted = self.hidden_layers[-1].get_activations_column
        return predicted - expected
                
        
    ##### UNDER CONSTRUCTION #################
    def train(self, transfer_func, df_train, df_validate):
        #Chequear si el numero de atributos concuerda con el numero de inputs especificados
        if(len(df_train.columns) != self.n_inputs):
            print('El número de columnas del dataset no coincide con el número de inputs especificado para esta red')
            print("Número de columnas del dataset: %d" %len(df_train.columns))
            print("Número de inputs aceptados por la red: %d" %self.n_inputs)
            return
        # extraer una fila y pasarsela a la funcion de forward_propagation
        
        
        
class Layer:
    def __init__(self, neurons_in_layer, neurons_previous_layer, index, **kargs):
        self.layer_index = index
        self.neurons_in_layer = neurons_in_layer
        self.activations_column = []
        
        #Funciones de transferencia disponibles para la capa
        if('transfer_func' in kargs):
            if(kargs['transfer_func'] == 'sigmoid'):
                self.transfer = self.sigmoid
                self.transfer_derivative = self.sigmoid_derivative
            elif(kargs['transfer_func'] == 'softmax'):
                self.transfer = self.softmax
                self.transfer_derivative = self.softmax_derivative
            elif(kargs['transfer_func'] == 'lineal' or index == 0): #Si es la capa de inputs le damos transferencia lineal
                self.transfer == self.linear
                self.transfer_derivative = selflinear_derivative
        else:# Si no se especifica una funcion de transferencia usaremos por defecto la sigmoidal
            self.transfer = self.sigmoid
            self.transfer_derivative = self.sigmoid_derivative
            
        #Si es la primera capa no tiene matriz de pesos
        if(self.layer_index == 0):
            self.weights_matrix = np.identity(self.neurons_in_layer)
            #Con esto logramos una matriz identidad que al ser multiplicada por el vector de inputs
            #nos da el mismo vector de inputs
        else:
            self.weights_matrix = np.random.rand(self.neurons_in_layer, neurons_previous_layer)
    
    def compute_layer_activations(self, activations_prev_layer):
        
        #Primero calculamos el input_sum, a la capa de input hay que simplemente pasarle la primera fila
        #de los datos como 'activations_prev_layer' cuando se entrene la red
        self.input_sum = np.dot(self.get_weights_matrix(), activations_prev_layer)
        
        #Ahora que calculamos el input sum, hay que pasar el resultado por la función de transferencia para ver
        #que es lo que las neuronas estan escupiendo
        
        self.activations_column = [self.transfer(activ_weights_sum) for activ_weights_sum in self.input_sum]
        #listo :V
        
        
    ###### Funciones de activacion #########
    
    # Sigmoid
    def sigmoid(self, activation):
        TOL = 1e-12 #La sigmoidal se agila en los extremos asi que hay que ponerle un limite de tolerancia
        if(activation > 0):
            activ = np.maximum(TOL, activation)
        elif(activation < 0):
            activ = np.maximum(-100,activation)
        return 1 / (1 + np.exp(-activ))
    def sigmoid_derivative(self, input_sum):
        return self.sigmoid(input_sum)(1.0 - self.sigmoid(input_sum))
    
    # SoftMax
    def softmax(self, activation):
        return exp(activation)/np.sum(map(exp, self.activations_column))
    def softmax_derivative(self, input_sum):
        return self.softmax(input_sum)(1.0 - self.softmax(input_sum))
        
    # Linear
    def linear(self, activation):
        return activation
    def linear_derivative(self, input_sum):
        return 1.0
    
    ########################################
    def get_parameters(self):
        layer_params = {'neurons_in_layer':self.neurons_in_layer, 'weights_matrix':self.weights_matrix, 
                        'index':self.layer_index, 'input_sum':self.input_sum, 
                        'activations_column':self.activations_column}
        return layer_params
    
    def get_weights_matrix(self):
        return self.weights_matrix
    
    def get_index(self):
        return self.layer_index
    
    def get_input_sum(self):
        return self.input_sum
    
    def get_activations_column(self):
        return self.activations_column
        
        


In [88]:
mynet = Network(5,(3,2), 1)

In [89]:

dict_net =  mynet.get_network_parameters()

In [90]:
layers = dict_net['hidden_layers']

In [104]:
layers[3].get_activations_column()

[array([ 0.5692296])]

In [92]:
a=layers[2].get_parameters()['weights_matrix']

In [93]:
data_row = np.ones((5,1))

In [97]:
mynet.forward_propagate(data_row)

In [50]:
c = [lineal(activation) for activation in b]
c[0]

array([ 0.44390954])